In [1]:
import numpy as np
import pandas as pd
import nltk

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# nltk.download("all")

In [2]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [3]:
# importar e concatenar os dados
df = pd.concat([pd.read_csv(__filename__) for __filename__ in ["COMENTARIOS_{}.csv".format(_) for _ in range(0, 2)]])

# corrigir outliers no score
print("Max score is: {}".format(df["score"].max()))

filter_matrix = df["score"] < df["score"].quantile(0.995)
df.loc[np.invert(filter_matrix), "score"] = df[filter_matrix]["score"].median()

print("Max score is: {}".format(df["score"].max()))

/home/insight/.local/share/virtualenvs/data-science--PSMU8hs/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Max score is: 9999999.0
Max score is: 150.0


In [4]:
print(len(df))

# removendo nulos
df = df.dropna()

print(len(df))

1010841
1010767


In [5]:
df["comment_wcount"] = df["comment"].str.count(" ") + 1
df["parent_comment_wcount"] = df["parent_comment"].str.count(" ") + 1

df["comment_char"] = df["comment"].str.len()
df["parent_comment_char"] = df["parent_comment"].str.len()

# df = df[0:100]
df.head()

,label,comment,author,subreddit,score,date,parent_comment,comment_wcount,parent_comment_wcount,comment_char,parent_comment_char
0,0,NC and NH.,Trumpbart,politics,2.0,2016-10,"Yeah, I get that argument. At this point, I'd ...",3,17,10,80
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4.0,2016-11,The blazers and Mavericks (The wests 5 and 6 s...,14,27,74,134
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3.0,2016-09,They're favored to win.,19,4,121,23
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8.0,2016-10,deadass don't kill my buzz,12,5,60,26
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6.0,2016-12,Yep can confirm I saw the tool they use for th...,7,18,31,85


In [6]:
def process_text(text):
    text = ''.join([c for c in text if c not in string.punctuation])
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    return [PorterStemmer().stem(word) for word in text]

# df['comment'].apply(process_text).head()

# wordcount_vetorizer = CountVectorizer(analyzer=process_text).fit(df['comment'])
# wordcount_vetorized = wordcount_vetorizer.transform(df['comment'])

# ml_df = pd.DataFrame(wordcount_vetorized.todense(), columns=wordcount_vetorizer.get_feature_names())
# ml_df = pd.concat([df['label'], ml_df], axis=1)

# tfidf_vetorizer = TfidfTransformer().fit(wordcount_vetorized)
# tfidf_df = pd.DataFrame(
#     tfidf_vetorizer.transform(wordcount_vetorized).todense(),
#     columns=wordcount_vetorizer.get_feature_names()
# )
# tfidf_df = pd.concat([df['label'], tfidf_df], axis=1)
# tfidf_df.head()

# display TF-IDF ranked words
# pd.DataFrame(tfidf_vetorizer.idf_, index=wordcount_vetorizer.get_feature_names(), columns=['TF-IDF']).sort_values(by=['TF-IDF'], ascending = False).head()

In [ ]:
# # Step - a : Remove blank rows if any.
# df['comment'].dropna(inplace=True)
# # Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
# df['comment'] = [entry.lower() for entry in df['comment']]
# # Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
# df['comment']= [word_tokenize(entry) for entry in df['comment']]
# # Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# # WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
# tag_map = defaultdict(lambda : wn.NOUN)
# tag_map['J'] = wn.ADJ
# tag_map['V'] = wn.VERB
# tag_map['R'] = wn.ADV
# for index,entry in enumerate(df['comment']):
#     # Declaring Empty List to store the words that follow the rules for this step
#     Final_words = []
#     # Initializing WordNetLemmatizer()
#     word_Lemmatized = WordNetLemmatizer()
#     # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
#     for word, tag in pos_tag(entry):
#         # Below condition is to check for Stop words and consider only alphabets
#         if word not in stopwords.words('english') and word.isalpha():
#             word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
#             Final_words.append(word_Final)
#     # The final processed set of words for each iteration will be stored in 'text_final'
#     df.loc[index,'comment'] = str(Final_words)

In [7]:
df['comment'] = df['comment'].astype(str)
df['label'] = df['label'].astype(int)

In [8]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['comment'],df['label'],test_size=0.3)
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['comment'])
Train_X = Tfidf_vect.transform(Train_X)
Test_X = Tfidf_vect.transform(Test_X)
print(Tfidf_vect.vocabulary_)
print(Train_X)

{'nc': 2981, 'and': 247, 'you': 4983, 'do': 1348, 'know': 2492, 'west': 4851, 'teams': 4403, 'play': 3321, 'against': 169, 'more': 2913, 'than': 4439, 'east': 1450, 'right': 3738, 'they': 4459, 'were': 4849, 'earlier': 1441, 'today': 4526, 'but': 687, 'since': 4033, 'announcement': 259, 'this': 4469, 'the': 4445, 'vegas': 4742, 'line': 2617, 'has': 2061, 'moved': 2928, 'to': 4525, 'patriots': 3220, 'meme': 2803, 'isn': 2371, 'funny': 1864, 'none': 3027, 'of': 3074, 'new': 3007, 'york': 4982, 'nigga': 3017, 'ones': 3106, 'are': 306, 'could': 1085, 'use': 4713, 'one': 3105, 'those': 4472, 'tools': 4543, 'don': 1363, 'pay': 3224, 'attention': 373, 'her': 2111, 'as': 333, 'long': 2662, 'she': 3959, 'legal': 2565, 'wouldn': 4944, 'kick': 2463, 'out': 3152, 'bed': 475, 'before': 479, 'took': 4541, 'load': 2644, 'trick': 4590, 'or': 3133, 'treating': 4584, 'in': 2269, 'general': 1895, 'is': 2366, 'just': 2442, 'weird': 4843, 'blade': 538, 'gtfo': 1996, 'have': 2071, 'good': 1953, 'build': 660

In [ ]:
# Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(tfidf_df.drop("label", axis=1),tfidf_df['label'],test_size=0.3)

In [9]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  65.99622070302838


In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)